# STGCN-PyTorch

## Packages

In [1]:
import random
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from load_data import *
from utils import *
from stgcn import *

## Random Seed

In [2]:
torch.manual_seed(2333)
torch.cuda.manual_seed(2333)
np.random.seed(2333)
random.seed(2333)
torch.backends.cudnn.deterministic = True

## Device

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## File Path

In [4]:
matrix_path = "dataset/W_228.csv"
data_path = "dataset/V_228.csv"
save_path = "save/model.pt"

## Parameters

In [5]:
day_slot = 288
n_train, n_val, n_test = 5,1,1

In [6]:
n_his = 12
n_pred = 3
n_route = 207
Ks, Kt = 3, 3
blocks = [[1, 32, 64],[1,32,64]]
drop_prob = 0

In [7]:
batch_size = 50
epochs = 50
lr = 1e-3

## Graph

In [8]:
W = load_matrix(matrix_path)
L = scaled_laplacian(W)
Lk = cheb_poly(L, Ks)
print(L.shape,Lk.shape)
Lk = torch.Tensor(Lk.astype(np.float32)).to(device)

(207, 207) (3, 207, 207)


## Standardization

In [9]:
train, val, test = load_data(data_path, n_train * day_slot, n_val * day_slot)
scaler = StandardScaler()
train = scaler.fit_transform(train)
val = scaler.transform(val)
test = scaler.transform(test)

## Transform Data

In [10]:
x_train, y_train = data_transform(train, n_his, n_pred, day_slot, device)
x_val, y_val = data_transform(val, n_his, n_pred, day_slot, device)
x_test, y_test = data_transform(test, n_his, n_pred, day_slot, device)
noise=0
x_train=x_train+(noise**0.5)*torch.randn(x_train.shape)
x_val=x_val+(noise**0.5)*torch.randn(x_val.shape)
x_test=x_test+(noise**0.5)*torch.randn(x_test.shape)
y_train=y_train+(noise**0.5)*torch.randn(y_train.shape)
y_val=y_val+(noise**0.5)*torch.randn(y_val.shape)
y_test=y_test+(noise**0.5)*torch.randn(y_test.shape)

## DataLoader

In [11]:
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)
test_data = torch.utils.data.TensorDataset(x_test, y_test)
test_iter = torch.utils.data.DataLoader(test_data,12)

## Loss & Model & Optimizer

In [12]:
loss = nn.MSELoss()
model = STGCN(Ks, Kt, blocks, n_his, n_route, Lk, drop_prob).to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)

## LR Scheduler

In [13]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

## Training & Save Model

In [ ]:
train_l=[]
val_l=[]
min_val_loss = np.inf
for epoch in range(1, epochs + 1):
    l_sum, n = 0.0, 0
    model.train()
    for x, y in train_iter:
        m = model(x)
#         print(m.shape)
        y_pred = m.view(len(x), -1)
#         print(y_pred.shape)
#         print(y.shape)
        l = loss(y_pred, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        l_sum += l.item() * y.shape[0]
        n += y.shape[0]
    scheduler.step()
    val_loss = evaluate_model(model, loss, val_iter)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), save_path)
    print("epoch", epoch, ", train loss:", l_sum / n, ", validation loss:", val_loss)
    train_l.append(l_sum/n)
    val_l.append(val_loss)

torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 64, 8, 207])
torch.Size([50, 1, 12, 207])
torch.Size([50

## Load Best Model

In [ ]:
best_model = STGCN(Ks, Kt, blocks, n_his, n_route, Lk, drop_prob).to(device)
best_model.load_state_dict(torch.load(save_path))

## Evaluation

In [ ]:
l = evaluate_model(best_model, loss, test_iter)
MAE, MAPE, RMSE,MAE_C, MAPE_C, RMSE_C,y_list,y_list_pred = evaluate_metric(best_model, test_iter, scaler)
print("test loss:", l, "\nMAE:", MAE, ", MAPE:", MAPE, ", RMSE:", RMSE, "\nMAE_C:", MAE_C, ", MAPE_C:", MAPE_C, ", RMSE_C:", RMSE_C)

In [ ]:
len(y_list_pred)
len(y_list_pred[-1])


In [ ]:
y=[]
s=40
for i in y_list:
    y.extend(i)
y=torch.tensor(y)

y=y.T
sensor_y=y[s]

y_pred=[]
for i in y_list_pred:
    y_pred.extend(i)
y_pred=torch.tensor(y_pred)
y_pred=y_pred.T
sensor_y_pred=y_pred[s]


In [ ]:
from matplotlib import pyplot as plt
plt.plot(len(sensor_y),sensor_y)
plt.plot(len(sensor_y_pred),sensor_y_pred,color='r')
plt.show()

In [ ]:
n = torch.stack([sensor_y,sensor_y_pred],dim=1)
n=n.numpy()
np.savetxt( "a.csv", n, delimiter="," )

In [ ]:
plt.plot(range(len(train_l)),train_l)
plt.plot(range(len(val_l)),val_l,color='r')
plt.show